# IMPORT

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import os

os.chdir(r'F:\\UNIVERSITY\\UNIVERSITY_DOCUMENTS\\CS231\\doan_v2')

from utils.utils_fn import save_json, load_json
from utils.utils_plt import bar_plot

from sklearn.preprocessing import MultiLabelBinarizer

from tqdm import tqdm
from collections import Counter

# **LOAD DATA**

In [2]:
data_path = 'data/train.json'
image_dir = 'images'
data = load_json(data_path)

In [13]:
len(df)

19525

In [3]:
# Data to DataFrame
data_dic = {
    'names': [val['name'] for val in data.values()],
    'genres': [val['genre'] for val in data.values()],
    'img_path': [os.path.join(image_dir, f'{key}.png') for key in data.keys()],
}

# To DataFrame
df = pd.DataFrame(data_dic)
df.head()

,names,genres,img_path
0,the remarkable life of ibelin,"[documentary, others]",images\000003.png
1,martha,[documentary],images\000007.png
2,what is a woman,"[comedy, documentary]",images\000012.png
3,no other land,[documentary],images\000014.png
4,never look away,"[documentary, others]",images\000018.png


In [4]:
# One Hot Encoding
mlb = MultiLabelBinarizer()
oh_genres = mlb.fit_transform(df['genres']) # Onehot Genres
classes = mlb.classes_

# Fit to df
df[classes] = oh_genres

# Create a df as copy of df
df_use = df.copy(deep=True)
df_use.drop(['genres', 'names'], inplace=True, axis=1)

In [5]:
df_use.head()

,img_path,action,adventure,comedy,crime,documentary,drama,fantasy,horror,mystery,others,romance,scifi,thriller
0,images\000003.png,0,0,0,0,1,0,0,0,0,1,0,0,0
1,images\000007.png,0,0,0,0,1,0,0,0,0,0,0,0,0
2,images\000012.png,0,0,1,0,1,0,0,0,0,0,0,0,0
3,images\000014.png,0,0,0,0,1,0,0,0,0,0,0,0,0
4,images\000018.png,0,0,0,0,1,0,0,0,0,1,0,0,0


# **TRAIN TEST SPLIT**


In [6]:
X = df_use['img_path'].values.reshape(-1, 1)
y = df_use[classes].values

In [7]:
from skmultilearn.model_selection import iterative_train_test_split
X_train, y_train, X_test, y_test = iterative_train_test_split(X, y, test_size=0.75)
# X_train, y_train, X_val, y_val = iterative_train_test_split(X_train, y_train, test_size=0.10)

# SAVE JSON

In [16]:
len(X_train)

4861

In [8]:
X_train_id = [os.path.basename(path.split('.')[0]) for path in X_train.flatten()]
# X_val_id = [os.path.basename(path.split('.')[0]) for path in X_val.flatten()]
X_test_id = [os.path.basename(path.split('.')[0]) for path in X_test.flatten()]

In [9]:
train_json = {key: val for key, val in data.items() if key in X_train_id}
# val_json = {key: val for key, val in data.items() if key in X_val_id}
test_json = {key: val for key, val in data.items() if key in X_test_id}

In [10]:
len(train_json)

4861

In [11]:
# Save path
train_path = 'data/train_new.json'
# val_path = 'data/val.json'
test_path = 'data/test.json'

# Save json
save_json(train_json, train_path)
# save_json(val_json, val_path)
# save_json(test_json, test_path)

# EDA

In [12]:
genres = [val['genre'] for val in test_json.values()]
genres = list(np.concatenate(genres))
Counter(genres)

Counter({'drama': 6340,
         'others': 5320,
         'thriller': 4623,
         'horror': 3458,
         'comedy': 3447,
         'romance': 3104,
         'documentary': 3083,
         'mystery': 2870,
         'scifi': 2597,
         'action': 2388,
         'crime': 2234,
         'adventure': 1586,
         'fantasy': 1251})